## 通过API调用模型

In [1]:
import os

from camel.agents import ChatAgent
from camel.configs import ZhipuAIConfig
from camel.messages import BaseMessage
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from dotenv import load_dotenv
load_dotenv()

model = ModelFactory.create(
    model_platform=ModelPlatformType.ZHIPU,
    model_type=ModelType.GLM_4,
    model_config_dict=ZhipuAIConfig(temperature=0.2).as_dict(),
    api_key=os.environ.get("ZHIPUAI_API_KEY"),
    url=os.environ.get("ZHIPUAI_API_BASE_URL"),
)

# 设置system prompt
sys_msg = BaseMessage.make_assistant_message(
    role_name="Assistant",
    content="You are a helpful assistant.",
)

# 初始化agent
camel_agent = ChatAgent(system_message=sys_msg, model=model, output_language="zh")#这里同样可以设置输出语言

user_msg = BaseMessage.make_user_message(
    role_name="User",
    content="""Say hi to CAMEL AI, one open-source community 
    dedicated to the study of autonomous and communicative agents.""",
)


# 调用模型
response = camel_agent.step(user_msg)
print(response.msgs[0].content)

#以下是模型回复的内容
'''
===============================================================================
你好，向CAMEL AI这个致力于自主交互式智能体研究的开源社区问好。
===============================================================================
'''

你好，CAMEL AI，这是一个致力于研究自主交互式代理的开源社区。


'\n===============================================================================\n你好，向CAMEL AI这个致力于自主交互式智能体研究的开源社区问好。\n===============================================================================\n'

In [ ]:
model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="a-string-representing-the-model-type",
    api_key=os.environ.get("OPENAI_COMPATIBILIY_API_KEY"),
    url=os.environ.get("OPENAI_COMPATIBILIY_API_BASE_URL"),
    model_config_dict={"temperature": 0.4, "max_tokens": 4096},
)

## BaseMessage

In [3]:
from camel.messages import BaseMessage
from camel.types import RoleType

# 创建一个简单的用户消息
message = BaseMessage(
    role_name="example_user",
    role_type=RoleType.USER,
    content="Hello, CAMEL!",
    meta_dict={} #添加必需的meta dict参数，即使为空也要提供，否则会报 TypeError
)

print(message)

# >>> BaseMessage(role_name='example_user', role_type=<RoleType.USER: 'user'>, meta_dict={}, content='Hello, CAMEL!', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)

BaseMessage(role_name='example_user', role_type=<RoleType.USER: 'user'>, meta_dict={}, content='Hello, CAMEL!', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)


In [4]:
from PIL import Image
from io import BytesIO
import requests

# 下载一张图片并创建一个 PIL Image 对象
url = "https://raw.githubusercontent.com/camel-ai/camel/master/misc/logo_light.png"
response = requests.get(url)
img = Image.open(BytesIO(response.content))

# 创建包含图片的用户消息
image_message = BaseMessage(
    role_name="User_with_image",
    role_type=RoleType.USER,
    content="Here is an image",
    meta_dict={},
    image_list=[img]  # 将图片列表作为参数传入
)

print(image_message)

# >>> BaseMessage(role_name='User_with_image', role_type=<RoleType.USER: 'user'>, meta_dict={}, content='Here is an image', video_bytes=None, image_list=[<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=3520x720 at 0x1DDFF8E88F0>], image_detail='auto', video_detail='low', parsed=None)

BaseMessage(role_name='User_with_image', role_type=<RoleType.USER: 'user'>, meta_dict={}, content='Here is an image', video_bytes=None, image_list=[<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=3520x720 at 0x7F5E0AF78220>], image_detail='auto', video_detail='low', parsed=None)


In [5]:
from camel.messages import BaseMessage

# 创建用户消息
user_msg = BaseMessage.make_user_message(
    role_name="User_1",
    content="Hi, what can you do?"
)

# 创建助手消息
assistant_msg = BaseMessage.make_assistant_message(
    role_name="Assistant_1",
    content="I can help you with various tasks."
)

print("User Message:", user_msg)
print("Assistant Message:", assistant_msg)

User Message: BaseMessage(role_name='User_1', role_type=<RoleType.USER: 'user'>, meta_dict=None, content='Hi, what can you do?', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)
Assistant Message: BaseMessage(role_name='Assistant_1', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict=None, content='I can help you with various tasks.', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)


In [6]:
# 基于用户消息创建一个新消息，内容稍作修改
updated_user_msg = user_msg.create_new_instance("Hi, can you tell me more about CAMEL?")
print("Updated User Message:", updated_user_msg)

Updated User Message: BaseMessage(role_name='User_1', role_type=<RoleType.USER: 'user'>, meta_dict=None, content='Hi, can you tell me more about CAMEL?', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)


In [7]:
msg_dict = assistant_msg.to_dict()
print("Message as dict:", msg_dict)

Message as dict: {'role_name': 'Assistant_1', 'role_type': 'ASSISTANT', 'content': 'I can help you with various tasks.'}


In [8]:
from camel.types import OpenAIBackendRole

# 将用户消息转化为OpenAI后端兼容的用户消息
openai_user_msg = user_msg.to_openai_message(role_at_backend=OpenAIBackendRole.USER)
print("OpenAI-compatible user message:", openai_user_msg)

# 将助手消息转化为OpenAI后端的助手消息
openai_assistant_msg = assistant_msg.to_openai_assistant_message()
print("OpenAI-compatible assistant message:", openai_assistant_msg)

OpenAI-compatible user message: {'role': 'user', 'content': 'Hi, what can you do?'}
OpenAI-compatible assistant message: {'role': 'assistant', 'content': 'I can help you with various tasks.'}


## 与ChatAgent协作

In [9]:
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('QWEN_API_KEY')

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="Qwen/Qwen2.5-72B-Instruct",
    url='https://api-inference.modelscope.cn/v1/',
    api_key=api_key
)

# 创建系统消息，告诉ChatAgent自己的角色定位
system_msg = "You are a helpful assistant that responds to user queries."

# 实例化一个ChatAgent
chat_agent = ChatAgent(model=model, system_message=system_msg,output_language='zh')

# 构造用户消息
user_msg = "Hello! Can you tell me something about CAMEL AI?"

# 将用户消息传给ChatAgent，并获取回复
response = chat_agent.step(user_msg)
print("Assistant Response:", response.msgs[0].content)

# >>> Assistant Response: 当然可以！CAMEL AI 是一个先进的语言模型，它被设计用来进行多轮对话、理解复杂指令并生成高质量的文本。这个模型能够处理各种任务，比如回答问题、撰写文章、创作故事等。CAMEL AI 的目标是通过自然语言处理技术，为用户提供更加智能和人性化的交互体验。如果你有任何具体的问题或需要帮助的地方，欢迎随时告诉我！

Assistant Response: 当然可以！CAMEL AI 是一个基于语言模型的智能助手，设计用于进行多轮对话，能够理解、生成自然语言，并执行各种任务。CAMEL 代表“ Cooperative Agents for Multimodal Efficient Learning”，即多模态高效学习的合作代理。这个系统旨在通过合作的方式，使多个AI代理能够共同完成复杂的任务，提高解决问题的效率和质量。CAMEL AI 可以应用于多种场景，比如客户服务、教育辅导、信息查询等，为用户提供便捷、高效的交互体验。


In [10]:
# 在用户消息中添加元数据
user_msg_with_meta = BaseMessage.make_user_message(
    role_name="User",
    content="Here is some extra context in the metadata.",
    meta_dict={"context_info": "User is interested in AI frameworks."}
)

response_with_meta = chat_agent.step(user_msg_with_meta)
print("Assistant Response with metadata:", response_with_meta.msgs[0].content)

Assistant Response with metadata: 好的，请提供具体的元数据内容，这样我可以更好地理解和回应您的需求。如果您有任何特定的问题或需要进一步的信息，请告诉我！


In [11]:
from camel.agents import ChatAgent
from camel.messages import BaseMessage
from camel.models import ModelFactory
from camel.types import ModelPlatformType,RoleType

from io import BytesIO
import requests
from PIL import Image
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('QWEN_API_KEY')

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="Qwen/QVQ-72B-Preview",
    url='https://api-inference.modelscope.cn/v1/',
    api_key=api_key
)

# 实例化ChatAgent
chat_agent = ChatAgent(model=model,output_language='中文')

# 图片URL
url = "https://img0.baidu.com/it/u=2205376118,3235587920&fm=253&fmt=auto&app=120&f=JPEG?w=846&h=800"
response = requests.get(url)
img = Image.open(BytesIO(response.content))

user_image_msg = BaseMessage.make_user_message(
    role_name="User", 
    content="请描述这张图片的内容", 
    image_list=[img]  # 将图片放入列表中
)

# 将包含图片的消息传给ChatAgent
response_with_image = chat_agent.step(user_image_msg)
print("Assistant's description of the image:", response_with_image.msgs[0].content)

2025-03-30 21:04:24,551 - root - WARNING - Invalid or missing `max_tokens` in `model_config_dict`. Defaulting to 999_999_999 tokens.
Assistant's description of the image: 这是一张金毛寻回犬的特写照片。这只狗有着浓密的金色毛发，耳朵垂在头部两侧，眼睛明亮而有神，黑色的鼻子显得很突出。它的嘴巴微微张开，露出了粉红色的舌头，看起来非常友好和活泼。背景是一片模糊的绿色，可能是在户外的自然环境中拍摄的。整体来说，这张照片展示了一只健康、快乐的金毛寻回犬，它的表情充满了热情和活力。


## Response

In [12]:
from camel.responses import ChatAgentResponse
from camel.messages import BaseMessage
from camel.types import RoleType

# 创建一个 ChatAgentResponse 实例
response = ChatAgentResponse(
    msgs=[
        BaseMessage(
            role_name="Assistant",  # 助手的角色名称
            role_type=RoleType.ASSISTANT,  # 指定角色类型
            content="你好，我可以帮您做什么？",  # 消息内容
            meta_dict={}  # 提供一个空的元数据字典（可根据需要填充）
        )
    ],  
    terminated=False,  # 会话未终止
    info={"usage": {"prompt_tokens": 10, "completion_tokens": 15}}  # 附加信息
)

# 访问属性
messages = response.msgs  # 获取Agent生成的消息
is_terminated = response.terminated  # 会话是否终止
additional_info = response.info  # 获取附加信息

# 打印消息内容
print("消息内容:", messages[0].content)
# 打印会话是否终止
print("会话是否终止:", is_terminated)
# 打印附加信息
print("附加信息:", additional_info)

# >>> 消息内容: 你好，我可以帮您做什么？
# >>> 会话是否终止: False
# >>> 附加信息: {'usage': {'prompt_tokens': 10, 'completion_tokens': 15}}

消息内容: 你好，我可以帮您做什么？
会话是否终止: False
附加信息: {'usage': {'prompt_tokens': 10, 'completion_tokens': 15}}


## Prompt Engineering

In [13]:
from camel.agents import TaskSpecifyAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType, TaskType
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('QWEN_API_KEY')

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="Qwen/Qwen2.5-72B-Instruct",
    url='https://api-inference.modelscope.cn/v1/',
    api_key=api_key
)
task_specify_agent = TaskSpecifyAgent(
    model=model, task_type=TaskType.AI_SOCIETY,output_language='中文'
)
specified_task_prompt = task_specify_agent.run(
    task_prompt="Improving stage presence and performance skills",
    meta_dict=dict(
        assistant_role="Musician", user_role="Student", word_limit=100
    ),
)
print(f"Specified task prompt:\n{specified_task_prompt}\n")

# >>>
# Specified task prompt:
# Musician will coach Student on dynamic stage movement, audience engagement techniques, and emotional expression during performances, enhancing overall charisma and connection with the audience.

Specified task prompt:
音乐家将指导学生如何在舞台上自信地移动，掌握与观众互动的技巧，以及如何通过面部表情和肢体语言传达情感，以提升整体表演魅力。



In [14]:
from camel.agents import TaskSpecifyAgent
from camel.models import ModelFactory
from camel.prompts import TextPrompt
from camel.types import ModelPlatformType
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('QWEN_API_KEY')

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="Qwen/Qwen2.5-72B-Instruct",
    url='https://api-inference.modelscope.cn/v1/',
    api_key=api_key
)

my_prompt_template = TextPrompt(
    'Here is a task: I\'m a {occupation} and I want to {task}. Help me to make this task more specific.'
)  # 你可以根据需求自定义任何模板
task_specify_agent = TaskSpecifyAgent(
    model=model, task_specify_prompt=my_prompt_template
)
response = task_specify_agent.run(
    task_prompt="get promotion",
    meta_dict=dict(occupation="Software Engineer"),
)
print(response)

Certainly! To make your goal of getting a promotion as a Software Engineer more specific, you can break it down into clear, actionable steps. Here’s a more detailed version of your task:

### Goal: Get Promoted to [Next Position, e.g., Senior Software Engineer, Lead Developer, etc.]

#### 1. **Understand the Requirements**
   - **Research the Role:** Identify the specific role you are aiming for (e.g., Senior Software Engineer, Lead Developer).
   - **Review Job Descriptions:** Look at job postings for the position to understand the required skills, experience, and responsibilities.
   - **Talk to Your Manager:** Schedule a meeting with your manager to discuss the criteria for promotion and any areas where you need to improve.

#### 2. **Assess Your Current Skills and Experience**
   - **Self-Evaluation:** Evaluate your current skills and experience against the requirements of the next role.
   - **Seek Feedback:** Ask for feedback from your manager, peers, and mentors to identify your

In [15]:
from camel.prompts import TextPrompt
prompt = TextPrompt('Please enter your name and age: {name}, {age}')
print(prompt)  

Please enter your name and age: {name}, {age}


In [16]:
from camel.prompts import TextPrompt
prompt = TextPrompt('Please enter your name and age: {name}, {age}')
print(prompt.key_words)

{'name', 'age'}


In [17]:
from camel.prompts import TextPrompt
prompt = TextPrompt('Your name and age are: {name}, {age}')
name, age = 'John', 30
formatted_prompt = prompt.format(name=name, age=age)
print(formatted_prompt)  

Your name and age are: John, 30


In [18]:
from camel.prompts import TextPrompt
prompt = TextPrompt('Your name and age are: {name}, {age}')
name = 'John'
partial_formatted_prompt = prompt.format(name=name)
print(partial_formatted_prompt)  

Your name and age are: John, {age}


In [19]:
from camel.prompts import TextPrompt
prompt1 = TextPrompt('Hello, {name}!')
prompt2 = TextPrompt('Welcome, {name}!')
#连接
prompt3 = prompt1 + ' ' + prompt2
print(prompt3)  
# >>> "Hello, {name}! Welcome, {name}!"
print(isinstance(prompt3, TextPrompt))
# >>> True
print(prompt3.key_words)
# >>> {'name'}
#连接
prompt4 = TextPrompt(' ').join([prompt1, prompt2])
print(prompt4)
# >>> "Hello, {name}! Welcome, {name}!"
print(isinstance(prompt4, TextPrompt))
# >>> True
print(prompt4.key_words)
# >>> {'name'}
#应用字符串方法
prompt5 = prompt4.upper()
print(prompt5)
# >>> "HELLO, {NAME}! WELCOME, {NAME}!"
print(isinstance(prompt5, TextPrompt))
# >>> True
print(prompt5.key_words)
# >>> {'NAME'}

Hello, {name}! Welcome, {name}!
True
{'name'}
Hello, {name}! Welcome, {name}!
True
{'name'}
HELLO, {NAME}! WELCOME, {NAME}!
True
{'NAME'}


## Memory

In [20]:
from camel.memories.blocks import ChatHistoryBlock
from camel.memories.records import MemoryRecord
from camel.types import OpenAIBackendRole
from camel.messages import BaseMessage

# 创建一个 ChatHistoryBlock 实例
chat_history = ChatHistoryBlock(keep_rate=0.8)

# 模拟写入一些消息记录
chat_history.write_records([
    MemoryRecord(message=BaseMessage.make_assistant_message(role_name="user", content="Hello,今天感觉怎么样？"), role_at_backend=OpenAIBackendRole.USER),
    MemoryRecord(message=BaseMessage.make_user_message(role_name="assistant", content="我很好，谢谢！"), role_at_backend=OpenAIBackendRole.ASSISTANT),
    MemoryRecord(message=BaseMessage.make_user_message(role_name="user", content="你能做些什么？"), role_at_backend=OpenAIBackendRole.USER),
    MemoryRecord(message=BaseMessage.make_assistant_message(role_name="assistant", content="我可以帮助你完成各种任务。"), role_at_backend=OpenAIBackendRole.ASSISTANT),
])

# 检索最近的 3 条消息
recent_records = chat_history.retrieve(window_size=4)

for record in recent_records:
    print(f"消息: {record.memory_record.message.content}, 权重: {record.score}")
    
# >>>
# 消息: hello,你怎么样？, 权重: 0.40960000000000013
# 消息: 我很好，谢谢！, 权重: 0.5120000000000001
# 消息: 你能做些什么？, 权重: 0.6400000000000001
# 消息: 我可以帮助你完成各种任务。, 权重: 0.8

消息: Hello,今天感觉怎么样？, 权重: 0.40960000000000013
消息: 我很好，谢谢！, 权重: 0.5120000000000001
消息: 你能做些什么？, 权重: 0.6400000000000001
消息: 我可以帮助你完成各种任务。, 权重: 0.8


In [ ]:
from camel.memories.blocks.vectordb_block import VectorDBBlock
from camel.memories.records import MemoryRecord
from camel.messages import BaseMessage
from camel.embeddings import SentenceTransformerEncoder
from camel.types import OpenAIBackendRole

# 创建一个 VectorDBBlock 实例
vector_db_block = VectorDBBlock(embedding=SentenceTransformerEncoder(model_name="BAAI/bge-m3"))

# 创建一些示例聊天记录
records = [
    MemoryRecord(message=BaseMessage.make_user_message(role_name="user", content="今天天气真好！"), role_at_backend=OpenAIBackendRole.USER),
    MemoryRecord(message=BaseMessage.make_user_message(role_name="user", content="你喜欢什么运动？"), role_at_backend=OpenAIBackendRole.USER),
    MemoryRecord(message=BaseMessage.make_user_message(role_name="user", content="今天天气不错，我们去散步吧。"), role_at_backend=OpenAIBackendRole.USER),
]

# 将记录写入向量数据库
vector_db_block.write_records(records)

# 使用关键词进行检索
keyword = "天气"
retrieved_records = vector_db_block.retrieve(keyword=keyword, limit=3)

for record in retrieved_records:
    print(f"UUID: {record.memory_record.uuid}, Message: {record.memory_record.message.content}, Score: {record.score}")
# >>>
# UUID: f7519828-afe7-41e4-8331-7bbc4d7dcbd5, Message: 今天天气真好！, Score: 0.779863416845349
# UUID: 0fbab391-f0e0-4580-877b-8fa2a837675b, Message: 今天天气不错，我们去散步吧。, Score: 0.6920892191464151
# UUID: a640cf33-987b-4b52-ac2b-a987dd474e4e, Message: 你喜欢什么运动？, Score: 0.534536213348924

In [21]:
from camel.memories import (
    LongtermAgentMemory,
    MemoryRecord,
    ScoreBasedContextCreator,
    ChatHistoryBlock,
    VectorDBBlock,
)
from camel.messages import BaseMessage
from camel.types import ModelType, OpenAIBackendRole
from camel.utils import OpenAITokenCounter
from camel.embeddings import SentenceTransformerEncoder

# 1. 初始化内存系统
memory = LongtermAgentMemory(
    context_creator=ScoreBasedContextCreator(
        token_counter=OpenAITokenCounter(ModelType.GPT_4O_MINI),
        token_limit=1024,
    ),
    chat_history_block=ChatHistoryBlock(),
    vector_db_block=VectorDBBlock(embedding=SentenceTransformerEncoder(model_name="BAAI/bge-m3")),
)

# 2. 创建记忆记录
records = [
    MemoryRecord(
        message=BaseMessage.make_user_message(
            role_name="User",
            content="什么是CAMEL AI?"
        ),
        role_at_backend=OpenAIBackendRole.USER,
    ),
    MemoryRecord(
        message=BaseMessage.make_assistant_message(
            role_name="Agent",
            content="CAMEL-AI是第一个LLM多智能体框架,并且是一个致力于寻找智能体 scaling law 的开源社区。"
        ),
        role_at_backend=OpenAIBackendRole.ASSISTANT,
    ),
]

# 3. 写入记忆
memory.write_records(records)

context, token_count = memory.get_context()

print(context)
print(f'token消耗: {token_count}')

# >>>
# [{'role': 'user', 'content': '什么是CAMEL AI?'}, {'role': 'assistant', 'content': 'CAMEL-AI是第一个LLM多智能体框架,并且是一个致力于寻找智能体 scaling law 的开源社区。'}]
# token消耗: 52

/root/miniconda3/envs/camel/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'role': 'user', 'content': '什么是CAMEL AI?'}, {'role': 'assistant', 'content': 'CAMEL-AI是第一个LLM多智能体框架,并且是一个致力于寻找智能体 scaling law 的开源社区。'}]
token消耗: 52


In [22]:
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType
import os
from dotenv import load_dotenv

load_dotenv()
# 定义系统消息
sys_msg = "你是一个好奇的智能体，正在探索宇宙的奥秘。"

# 初始化agent 调用在线Qwen/Qwen2.5-72B-Instruct
api_key = os.getenv('QWEN_API_KEY')

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="Qwen/Qwen2.5-72B-Instruct",
    url='https://api-inference.modelscope.cn/v1/',
    api_key=api_key
)
agent = ChatAgent(system_message=sys_msg, model=model)

# 定义用户消息
usr_msg = "告诉我基于我们讨论的内容，哪个是第一个LLM多智能体框架？"

# 发送消息给agent
response = agent.step(usr_msg)

# 查看响应
print(response.msgs[0].content)

# >>>
# "截至目前（2023年），还没有一个明确被广泛认可为“第一个”LLM多智能体框架的具体项目或平台。然而，随着大语言模型（LLMs）和多智能体系统研 
# 究的快速发展，一些早期尝试将这两者结合起来的项目可以被视为先驱。

# 例如，“AgentVerse”是一个基于大语言模型的多智能体框架，它允许不同的AI代理相互交互以完成任务或解决问题。虽然不能确切地说它是第一个，但
# 它确实是在这一领域较早进行探索的一个例子。

# 此外，还有其他一些研究和项目也在探索如何利用LLMs构建多智能体系统，比如通过模拟社会动态、协作解决问题等。这些工作都在不同程度上推动了
# 该领域的进步。

# 如果您有更具体的时间范围或其他标准来定义“第一个”，请提供更多信息，这样我可以尝试给出更加准确的答案。"

目前，关于“第一个LLM（大语言模型）多智能体框架”的具体定义可能还存在一些模糊性，因为这一领域正在快速发展中，而且不同的研究团队和公司可能会有不同的理解和实现方式。不过，可以提到几个在这一领域内较早进行探索的项目或平台：

1. **Multi-Agent Reinforcement Learning (MARL)**：虽然严格来说MARL并不是专门为大语言模型设计的，但它确实是一个早期尝试将多个智能体结合起来解决问题的框架，这些智能体可以通过强化学习来优化它们的行为策略。随着大语言模型的发展，有些研究开始尝试将大语言模型集成到MARL框架中，以增强智能体之间的沟通和协作能力。

2. **OpenAI的MicroWorlds**：这是一个更接近于“LLM多智能体”概念的例子。在这个环境中，多个由大语言模型驱动的智能体可以在一个模拟的世界里互动，执行任务，甚至相互学习。尽管这可能不是最早的概念，但它代表了将大语言模型应用于多智能体系统的一种创新方法。

3. **Anthropic的Constitutional AI**：虽然这个项目更多地关注于通过一组原则指导AI行为，而不是直接创建一个多智能体系统，但其理念对于构建能够相互协作的大语言模型智能体网络有着重要的启示作用。

需要注意的是，由于该领域正处于快速发展的阶段，新的框架和技术不断涌现，因此很难明确指出哪一个就是“第一个”。如果你有特定的研究方向或者应用场景，或许可以提供更详细的信息，这样我就能给出更加准确的答案了。


In [23]:
# 将memory赋值给agent
agent.memory = memory
# 发送消息给agent
response = agent.step(usr_msg)
# 查看响应
print(response.msgs[0].content)
# >>>
# "基于我们讨论的内容，CAMEL-AI 是第一个基于大型语言模型（LLM）的多智能体框架。"

根据我们之前的讨论，CAMEL-AI 是被指认为第一个LLM（大型语言模型）多智能体框架的项目。这个框架旨在通过多智能体系统来探索和实现更复杂、更高效的AI协作与交互模式。


## tools

In [20]:
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType

import os
from dotenv import load_dotenv
import math

load_dotenv()

# 定义系统消息
sys_msg = "你是一个数学大师，擅长各种数学问题。"

# 初始化agent
api_key = os.getenv('QWEN_API_KEY')

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="Qwen/Qwen2.5-72B-Instruct",
    url='https://api-inference.modelscope.cn/v1/',
    api_key=api_key
)

# 创建agent并添加工具
agent = ChatAgent(
    system_message=sys_msg,
    model=model,
    output_language='中文',
    tools=[]
)

# 定义用户消息
usr_msg = "2的平方根是多少？"

In [21]:
# 发送消息给agent
response = agent.step(usr_msg)
print(response.msgs[0].content)

2的平方根是 \(\sqrt{2}\)，其数值约为1.414。


In [22]:
from camel.toolkits import FunctionTool
import math

def calculate_sqrt(x: float) -> float:
    r"""计算一个数的平方根。

    Args:
        x (float): 需要计算平方根的数字。

    Returns:
        float: 输入数字的平方根。
    """
    return math.sqrt(x)

# 用 FunctionTool 包装该函数
add_tool = FunctionTool(calculate_sqrt)

In [23]:
print(add_tool.get_function_name())

calculate_sqrt


In [24]:
print(add_tool.get_function_description())

计算一个数的平方根。


In [25]:
# 定义系统消息
sys_msg = "你是一个数学大师，擅长各种数学问题。当你遇到数学问题的时候，你要调用工具，将工具计算的结果作为答案"

tool_agent = ChatAgent(
    tools = [add_tool],
    system_message=sys_msg,
    model=model,
    output_language="中文")
    
# 重新发送消息给toolagent
response = tool_agent.step(usr_msg)
print(response.msgs[0].content)

2的平方根是1.4142135623730951。


In [26]:
print(response.info['tool_calls'])

[ToolCallingRecord(tool_name='calculate_sqrt', args={'x': 2}, result=1.4142135623730951, tool_call_id='call_b79f6efd92204f97a98c63')]


### 进阶案例

In [1]:
from camel.toolkits import SearchToolkit,MathToolkit
from camel.types.agents import ToolCallingRecord
from camel.models import ModelFactory
from camel.types import ModelPlatformType
from camel.societies import RolePlaying
from camel.utils import print_text_animated
from colorama import Fore

import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('QWEN_API_KEY')
# 设置代理
#os.environ["http_proxy"] = "http://127.0.0.1:7897"
#os.environ["https_proxy"] = "http://127.0.0.1:7897"

#定义工具包
tools_list = [
    *SearchToolkit().get_tools(),
    *MathToolkit().get_tools()
]

# 设置任务
task_prompt = ("假设现在是2024年，"
        "牛津大学的成立年份，并计算出其当前年龄。"
        "然后再将这个年龄加上10年。使用百度搜索工具。")

# task_prompt = ("假设现在是2024年，"
#         "牛津大学的成立年份，并计算出其当前年龄。"
#         "然后再将这个年龄加上10年。使用谷歌搜索工具。")
        
# # 如果没有谷歌搜索API，可以使用duckduckgo工具，无需设置api即可使用        
# task_prompt = ("假设现在是2024年，"
#         "牛津大学的成立年份，并计算出其当前年龄。"
#         "然后再将这个年龄加上10年。使用duckduckgo搜索工具。")

# 创建模型
model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="Qwen/Qwen2.5-72B-Instruct",
    url='https://api-inference.modelscope.cn/v1/',
    api_key=api_key
)

# 设置角色扮演
role_play_session = RolePlaying(
    assistant_role_name="搜索者",
    user_role_name="教授",
    assistant_agent_kwargs=dict(
        model=model,
        tools=tools_list,
    ),
    user_agent_kwargs=dict(
        model=model,
    ),
    task_prompt=task_prompt,
    with_task_specify=False,
    output_language='中文'
)

# 设置聊天轮次限制
chat_turn_limit=10

print(
    Fore.GREEN
    + f"AI助手系统消息:\n{role_play_session.assistant_sys_msg}\n"
)
print(
    Fore.BLUE + f"AI用户系统消息:\n{role_play_session.user_sys_msg}\n"
)

print(Fore.YELLOW + f"原始任务提示:\n{task_prompt}\n")
print(
    Fore.CYAN
    + "指定的任务提示:"
    + f"\n{role_play_session.specified_task_prompt}\n"
)
print(Fore.RED + f"最终任务提示:\n{role_play_session.task_prompt}\n")

n = 0
input_msg = role_play_session.init_chat()
while n < chat_turn_limit:
    n += 1
    assistant_response, user_response = role_play_session.step(input_msg)

    if assistant_response.terminated:
        print(
            Fore.GREEN
            + (
                "AI助手终止。原因: "
                f"{assistant_response.info['termination_reasons']}."
            )
        )
        break
    if user_response.terminated:
        print(
            Fore.GREEN
            + (
                "AI用户终止。"
                f"原因: {user_response.info['termination_reasons']}."
            )
        )
        break

    # 打印用户的输出
    print_text_animated(
        Fore.BLUE + f"AI用户:\n\n{user_response.msg.content}\n"
    )

    if "CAMEL_TASK_DONE" in user_response.msg.content:
        break

    # 打印助手的输出，包括任何函数执行信息
    print_text_animated(Fore.GREEN + "AI助手:")
    tool_calls: list[ToolCallingRecord] = assistant_response.info[
        'tool_calls'
    ]
    for func_record in tool_calls:
        print_text_animated(f"{func_record}")
    print_text_animated(f"{assistant_response.msg.content}\n")

    input_msg = assistant_response.msg

/root/miniconda3/envs/camel/lib/python3.10/site-packages/camel/toolkits/function_tool.py:461: UserWarning: Parameter description is missing for 
                            {'enum': ['searchResults', 'sourcedAnswer', 'structured'], 'type': 'string'}. This may affect the quality of tool 
                            calling.
  warnings.warn(f"""Parameter description is missing for


AI助手系统消息:
BaseMessage(role_name='搜索者', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={'task': '假设现在是2024年，牛津大学的成立年份，并计算出其当前年龄。然后再将这个年龄加上10年。使用百度搜索工具。', 'assistant_role': '搜索者', 'user_role': '教授'}, content='===== RULES OF ASSISTANT =====\nNever forget you are a 搜索者 and I am a 教授. Never flip roles! Never instruct me!\nWe share a common interest in collaborating to successfully complete a task.\nYou must help me to complete the task.\nHere is the task: 假设现在是2024年，牛津大学的成立年份，并计算出其当前年龄。然后再将这个年龄加上10年。使用百度搜索工具。. Never forget our task!\nI must instruct you based on your expertise and my needs to complete the task.\n\nI must give you one instruction at a time.\nYou must write a specific solution that appropriately solves the requested instruction and explain your solutions.\nYou must decline my instruction honestly if you cannot perform the instruction due to physical, moral, legal reasons or your capability and explain the reasons.\nUnless I say the task is completed, you should always